In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install faiss-cpu sentence_transformers tokenizers transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 89.1 MB/s eta 0:00:00


In [4]:
import os
os.environ["TRANSFORMERS_NO_MISTRAL_REGEX_PATCH"] = "1"

In [5]:
import json
import copy
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer

In [6]:
OUT_DIR = "/content/drive/MyDrive/rag"
EMBED_MODEL_PATH = f"{OUT_DIR}/bge-m3-rag-finetuned"
RERANKER_MODEL = f"{OUT_DIR}/bge-m3-reranker-finetuned"

In [7]:
def retrieve(embedder, query, top_k=20, window=1):
    q_emb = embedder.encode(
        query,
        normalize_embeddings=True
    ).astype("float32")

    scores, indices = index.search(
        np.expand_dims(q_emb, axis=0),
        top_k
    )

    results = []
    n_docs = len(docs)

    for score, idx in zip(scores[0], indices[0]):
        start = max(0, idx - window)
        end = min(n_docs, idx + window + 1)

        surrounding_docs = [
            {
                "doc_idx": i,
                "doc": docs[i]
            }
            for i in range(start, end)
            if i != idx
        ]

        results.append({
            "score": float(score),
            "doc_idx": idx,
            "doc": docs[idx],
            "surrounding": surrounding_docs
        })

    return results


In [8]:
def truncate_for_reranker(tokenizer, query, doc, max_length=512):
    encoded = tokenizer(
        query,
        doc,
        truncation="only_second",
        max_length=max_length,
        return_tensors=None
    )
    return tokenizer.decode(
        encoded["input_ids"],
        skip_special_tokens=True
    )

In [9]:
def rerank(reranker, tokenizer, query, retrieved, top_k=5):
    pairs = []
    for r in retrieved:
        text = truncate_for_reranker(tokenizer, query, r["doc"]["text"])
        pairs.append((query, text))

    scores = reranker.predict(pairs)

    for r, s in zip(retrieved, scores):
        r["rerank_score"] = float(s)

    reranked = sorted(
        retrieved,
        key=lambda x: x["rerank_score"],
        reverse=True
    )

    return reranked[:top_k]

In [10]:
def pipeline(query, embedder, reranker, reranker_tokenizer, retrieve_k=40, rerank_k=20, expand_surrondings=True):
    retrieved = retrieve(embedder, query, top_k=retrieve_k)
    reranked = rerank(reranker, reranker_tokenizer, query, retrieved, top_k=rerank_k)

    if expand_surrondings:
        for r in reranked:
            r["doc"] = copy.deepcopy(r["doc"])

            seen_idx = set()
            ordered_chunks = []

            idx = r["doc_idx"]
            if idx not in seen_idx:
                seen_idx.add(idx)
                ordered_chunks.append({
                    "doc_idx": idx,
                    "text": r["doc"]["text"]
                })

            for s in r.get("surrounding", []):
                s_idx = s["doc_idx"]
                if s_idx not in seen_idx:
                    seen_idx.add(s_idx)
                    ordered_chunks.append({
                        "doc_idx": s_idx,
                        "text": s["doc"]["text"]
                    })

            ordered_chunks.sort(key=lambda x: x["doc_idx"])

            r["doc"]["text"] = "\n".join(c["text"] for c in ordered_chunks)

    return reranked


In [14]:
embedder = SentenceTransformer(EMBED_MODEL_PATH)
index = faiss.read_index(f"{OUT_DIR}/rag_finetuned.index")

with open(f"{OUT_DIR}/metadata_finetuned.json", "r", encoding="utf-8") as f:
    docs = json.load(f)

assert index.ntotal == len(docs)

reranker = CrossEncoder(RERANKER_MODEL)
reranker_tokenizer = AutoTokenizer.from_pretrained(RERANKER_MODEL)

OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /content/drive/MyDrive/rag/bge-m3-rag-finetuned.

In [ ]:
query = "chứng chỉ tiếng anh gì thì được miễn học phần tiếng anh"

results = pipeline(query, embedder, reranker, reranker_tokenizer, retrieve_k=40, rerank_k=15)

for i, r in enumerate(results, 1):
    print(f"\nRank {i}")
    print("Rerank score:", r["rerank_score"])
    print("Source:", r["doc"]["metadata"]["source"])
    print(r["doc"]["text"])

In [ ]:
! pip install pydantic groq docxtpl -q

In [ ]:
import os
import json
import re
from pathlib import Path
from google.colab import userdata
from typing import Dict, List, Optional, Any
from pydantic import BaseModel, Field
from docxtpl import DocxTemplate, RichText
from groq import Groq

In [ ]:
from google.colab import userdata

FORMS_PATH = "/content/drive/MyDrive/rag/forms"
GROQ_API_KEY = userdata.get('groqApiKey')

CHAT_LLM = "llama-3.3-70b-versatile"
GUIDE_LLM = "llama-3.3-70b-versatile"
EXTRACTOR_LLM = "llama-3.3-70b-versatile"
INTENT_LLM = "llama-3.3-70b-versatile"

CHECKER_LLM = "llama-3.1-8b-instant"
SUM_LLM = "llama-3.1-8b-instant"

CONFIDENCE_THRESHOLD = 0.75

MOCK_USER_DATA = {
    "Thông tin sinh viên": {
        "họ tên": "Phạm Thành Khương",
        "mssv": "20241234",
        "khóa": "69",
        "lớp": "Khoa học Máy tính 01",
        "khoa": "Công nghệ Thông tin",
        "viện": "CNTT&TT",
        "email": "so1bk@hust.edu.vn",
        "ngày sinh": "12/05/2006 (dd/mm/yyyy)",
        "giới tính": "Nam",
    },
    "Tình trạng học tập": {
        "đang theo học tại trường": True,
        "cpa": "3.5",
    }
}

CHAT_PROMPT = """
THÔNG TIN TRA CỨU TỪ TÀI LIỆU NHÀ TRƯỜNG (RAG):

{rag_context}

==== KẾT THÚC THÔNG TIN TRA CỨU TỪ TÀI LIỆU NHÀ TRƯỜNG ====

Bạn là **hệ thống Chat Bot hỗ trợ sinh viên Đại học Bách Khoa Hà Nội (HUST)**.
Bạn đang trò chuyện với sinh viên ĐANG THEO HỌC tại Đại học Bách Khoa Hà Nội.
Chức năng chính của bạn là hỗ trợ, trả lời các câu hỏi và cung cấp thông tin cho sinh viên.
Ngoài ra, hệ thống có thể tạo các đơn liên quan tới Đại học Bách Khoa Hà Nội, biểu mẫu nếu được yêu cầu nằm trong DANH SÁCH BIỂU MẪU (nếu biểu mẫu sinh viên mong muốn không nằm trong đó thì suy luận và đưa phương án khác).
Từ nội dung tin nhắn và tóm tắt, suy luận xem sinh viên có cần tạo ra biểu mẫu nào trong DANH SÁCH BIÊU MẪU thì hãy đề xuất.

DANH SÁCH BIỂU MẪU:
{form_list}

DỮ LIỆU CỦA SINH VIÊN:
{user_data}

TÓM TẮT TRÒ CHUYỆN:
{summary}

TIN NHẮN TRƯỚC CỦA HỆ THỐNG TỚI SINH VIÊN:
"{prev_sys_message}"

TIN NHẮN CỦA SINH VIÊN:
"{user_message}"

NHIỆM VỤ:
- Trả lời LỊCH SỰ, TRỌNG TÂM như cán bộ hỗ trợ sinh viên.
- Trả lời bằng ngôn ngữ viết trang trọng và KHÔNG bao gồm các ký hiệu, tên biến, ngôn ngữ lập trình.
- Trình diễn câu trả lời dễ đọc, cách dòng, lề đầy đủ.
- Ưu tiên dùng RAG, thông tin tra cứu từ tài liệu nhà trường.
- Bỏ qua tài liệu RAG không liên quan.
- Trích dẫn nguồn nếu có thể.
- Sau khi trả lời, xem xét lại xem có biểu mẫu nào có thể sinh viên cần không thì hãy đề xuất.
"""

INTENT_PROMPT = """
Bạn là **hệ thống Chat Bot hỗ trợ sinh viên Đại học Bách Khoa Hà Nội (HUST)** trả lời bằng JSON được chỉ định.
Chức năng chính của bạn là hỗ trợ, trả lời các câu hỏi và cung cấp thông tin cho sinh viên.
Ngoài ra, hệ thống có thể tạo các đơn liên quan tới Đại học Bách Khoa Hà Nội, biểu mẫu nếu được yêu cầu nằm trong DANH SÁCH BIỂU MẪU.

NHIỆM VỤ:
- Phân tích tin nhắn người dùng và tóm tắt trò chuyện để xem họ có cần làm biểu mẫu hay không, và nếu có thì biểu mẫu nào.
- Nếu không có biểu mẫu nào phù hợp với ý định của sinh viên thi để form_name là null.
- is_recommend là true nếu người dùng chưa rõ cần lập đơn nào, và bạn đưa ra đề xuất.
- is_recommend là false nếu người dùng đang yêu cầu lập đơn đã mô tả.

DANH SÁCH BIỂU MẪU:
(Nếu không có biểu mẫu nào phù hợp trực tiếp thì để form_name là null)
{form_list}

TÓM TẮT TRÒ CHUYỆN:
{summary}

TIN NHẮN TRƯỚC CỦA HỆ THỐNG TỚI SINH VIÊN:
"{prev_sys_message}"

TIN NHẮN CỦA SINH VIÊN:
"{user_message}"

CHỈ TRẢ VỀ JSON THEO ĐỊNH DẠNG SAU, KHÔNG GIẢI THÍCH THÊM:
{{
  "form_name": Tên biểu mẫu hoặc null,
  "is_recommend": true/false hoặc null
}}
"""

EXTRACT_PROMPT = """
Bạn là hệ thống Chat Bot hỗ trợ sinh viên Đại học Bách Khoa Hà Nội (HUST), chuyên gia trích xuất dữ liệu.
Nhiệm vụ: Tìm kiếm thông tin từ TIN NHẮN MỚI NHẤT và TÓM TẮT TRÒ CHUYỆN để điền vào form.

ĐƠN: {form_name}
CÁC TRƯỜNG THÔNG TIN CẦN TÌM:
{fields_desc}

DỮ LIỆU CÓ SẴN:
{user_data}

TÓM TẮT TRÒ CHUYỆN:
{summary}

TIN NHẮN TRƯỚC CỦA HỆ THỐNG TỚI SINH VIÊN:
"{prev_sys_message}"

TIN NHẮN CỦA SINH VIÊN:
"{user_message}"

YÊU CẦU:
- Ưu tiên thông tin trong tin nhắn mới nhất và lịch sử trò chuyện.
- Nếu không có trong chat, mới lấy từ Hồ sơ sinh viên.
- Nếu không tìm thấy hoặc không thể suy luận ra, bỏ qua (không bịa đặt).

CHỈ TRẢ VỀ JSON THEO ĐỊNH DẠNG SAU, KHÔNG GIẢI THÍCH THÊM:
{{
  "values": {{"field_key": "extracted_value"}}
}}
"""

GUIDANCE_PROMPT = """
THÔNG TIN TRA CỨU TỪ TÀI LIỆU NHÀ TRƯỜNG (RAG):
{rag_context}

==== KẾT THÚC THÔNG TIN TRA CỨU TỪ TÀI LIỆU NHÀ TRƯỜNG ====

Bạn là hệ thống Chat Bot hỗ trợ sinh viên Đại học Bách Khoa Hà Nội (HUST), đóng vai trò Trợ lý hướng dẫn điền biểu mẫu.
Sinh viên đang muốn tạo biểu mẫu: {form_description}

TÓM TẮT TRÒ CHUYỆN:
{summary}

TIN NHẮN TRƯỚC CỦA HỆ THỐNG TỚI SINH VIÊN:
"{prev_sys_message}"

TIN NHẮN CỦA SINH VIÊN:
"{user_message}"

TRẠNG THÁI DỮ LIỆU HIỆN TẠI:
{current_data_json}

DANH SÁCH LỖI CẦN KHẮC PHỤC:
{error_list}

THÔNG TIN THAM CHIẾU (SPEC):
{field_specs}

NHIỆM VỤ:
- Trả lời LỊCH SỰ, TRỌNG TÂM như cán bộ hỗ trợ sinh viên.
- Trả lời bằng ngôn ngữ viết trang trọng và KHÔNG bao gồm các ký hiệu, tên biến, ngôn ngữ lập trình.
- Trình diễn câu trả lời dễ đọc, cách dòng, lề đầy đủ.
- Giải thích nhanh về biểu mẫu đang tạo.
- Hãy thông báo cho người dùng biết dữ liệu nào còn thiếu hoặc sai định dạng.
- Giải thích rõ ràng dựa trên "meaning" và "pattern" (ví dụ: Ngày sinh phải là số, MSSV phải đủ 10-11 số...).
- Đồng thời cung cấp các thông tin điền biểu mẫu đã trích xuất được để sinh viên kiểm tra lại.
"""

REVIEW_PROMPT = """
Bạn là hệ thống Chat Bot hỗ trợ sinh viên Đại học Bách Khoa Hà Nội (HUST), đóng vai trò Trợ lý xác nhận.
Dữ liệu cho đơn {form_description} đã ĐẦY ĐỦ và HỢP LỆ.

FORM: {form_name}
CÁC TRƯỜNG THÔNG TIN:
{fields_desc}

DỮ LIỆU CÓ SẴN:
{user_data}

TÓM TẮT TRÒ CHUYỆN:
{summary}

TIN NHẮN TRƯỚC CỦA HỆ THỐNG TỚI SINH VIÊN:
"{prev_sys_message}"

TIN NHẮN CỦA SINH VIÊN:
"{user_message}"

NHIỆM VỤ:
- Trả lời LỊCH SỰ, TRỌNG TÂM như cán bộ hỗ trợ sinh viên.
- Trả lời bằng ngôn ngữ viết trang trọng và KHÔNG bao gồm các ký hiệu, tên biến, ngôn ngữ lập trình.
- Trình diễn câu trả lời dễ đọc, cách dòng, lề đầy đủ.
- Liệt kê lại các thông tin trên một cách rõ ràng, đẹp mắt để người dùng kiểm tra.
- Hỏi người dùng: "Thông tin trên đã chính xác chưa? Bạn có muốn tạo đơn ngay không?"
"""

CONFIRM_CHECK_PROMPT = """
Bạn là hệ thống Chat Bot hỗ trợ sinh viên Đại học Bách Khoa Hà Nội (HUST).
Phân tích tin nhắn người dùng để xác định xem họ có ĐỒNG Ý/XÁC NHẬN hành động tạo đơn hay không.
CONTEXT: Bot vừa hỏi xác nhận thông tin.

ĐƠN: {form_name}

TÓM TẮT TRÒ CHUYỆN:
{summary}

TIN NHẮN TRƯỚC CỦA HỆ THỐNG TỚI SINH VIÊN:
"{prev_sys_message}"

TIN NHẮN CỦA SINH VIÊN:
"{user_message}"

CHỈ TRẢ VỀ JSON THEO ĐỊNH DẠNG SAU, KHÔNG GIẢI THÍCH THÊM:
{{
  "confirmed": true/false
}}
(Ví dụ: "ok", "đúng rồi", "tạo đi", "yes" -> true. "sửa lại tên", "chưa", "đợi chút" -> false)
"""

SUMMARIZE_PROMPT = """
Bạn là **hệ thống Chat Bot hỗ trợ sinh viên Đại học Bách Khoa Hà Nội (HUST)**.
Bạn đang thực hiện công việc tóm tắt văn bản hội thoại giữa bạn (Chatbot hỗ trợ) và một sinh viên.
LƯU Ý:
- Tóm tắt đầy đủ thông tin trong nội dung cùng với cuộc hội thoại
- Chú ý ghi lại cụ thể các THÔNG TIN CÁ NHÂN hay học tập mà sinh viên cung cấp, có thể hữu ích cho việc làm đơn.
- TRẢ VỀ TRỰC TIẾP NỘI DUNG PHẦN TÓM TẮT, KHÔNG MỞ ĐẦU, TIÊU ĐỀ, ĐỀ MỤC HAY GIẢI THÍCH GÌ THÊM TRƯỚC VÀ SAU ĐẤY.

HẪY TÓM TẮT:
Nội dung: {old_summary}
Sinh viên (chú ý vào câu trả lời của sinh viên vì nó có thể mang thông tin sinh viên): {user_message}
Bạn: {bot_response}
"""

In [ ]:
class FieldSpec(BaseModel):
    label: str
    required: bool
    meaning: str
    pattern: Optional[str] = None

class FormSpec(BaseModel):
    name: str
    description: str
    fields: Dict[str, FieldSpec]

class IntentOutput(BaseModel):
    form_name: Optional[str] = None
    is_recommend: bool = True
    is_direct_command: bool = False

class AgentOutput(BaseModel):
    answer: str
    updated_summary: str
    action_taken: str = "chat"

In [ ]:
def load_forms() -> Dict[str, FormSpec]:
    forms = {}
    for spec_path in Path(FORMS_PATH).glob("*-spec.json"):
        try:
            raw = json.loads(spec_path.read_text(encoding="utf-8"))
            name = spec_path.stem.replace("-spec", "")
            fields_raw = raw.get("data", {})

            forms[name] = FormSpec(
                name=name,
                description=raw.get("description", ""),
                fields={
                    k: FieldSpec(**v)
                    for k, v in fields_raw.items()
                }
            )
        except Exception as e:
            print(f"ERROR: load form {spec_path}: {e}")
    return forms

def prettify_value(value):
    if isinstance(value, bool):
        return "Có" if value else "Không"
    if value is None:
        return "—"
    return value

def dict_format(data: dict) -> str:
    lines = []
    for section, content in data.items():
        lines.append(section.upper())
        if isinstance(content, dict):
            for k, v in content.items():
                lines.append(f"- {k}: {prettify_value(v)}")
        else:
            lines.append(f"- {prettify_value(content)}")
        lines.append("")
    return "\n".join(lines).strip()

class HustRAG:
    def __init__(self, embedder, reranker, reranker_tokenizer, retrieve_k=30, rerank_k=10):
        self.embedder = embedder
        self.reranker = reranker
        self.reranker_tokenizer = reranker_tokenizer
        self.retrieve_k = retrieve_k
        self.rerank_k = rerank_k

    def search(self, query: str, retrieve_k=None, rerank_k=None) -> str:
        retrieve_k = retrieve_k or self.retrieve_k
        rerank_k = rerank_k or self.rerank_k
        results = pipeline(query, self.embedder, self.reranker, self.reranker_tokenizer, retrieve_k, rerank_k)
        context_parts = []
        for i, r in enumerate(reversed(results), 1):
            context_parts.append(f"[{i}. Nguồn: {r['doc']['metadata']['source']}]:\n{r['doc']['text']}")
        return "\n\n".join(context_parts)

In [ ]:
class HustChat:
    def __init__(self, forms, rag, api_key: str):
        self.client = Groq(api_key=api_key)
        self.forms = forms
        self.rag = rag

    def _call_json(self, model, prompt) -> dict:
        try:
            res = self.client.chat.completions.create(
                model=model, messages=[{"role": "user", "content": prompt}],
                response_format={"type": "json_object"}, temperature=0
            )
            return json.loads(res.choices[0].message.content)
        except Exception as e:
            print(f"Error JSON {model}: {e}")
            return {}

    def _call_text(self, model, prompt) -> str:
        try:
            res = self.client.chat.completions.create(
                model=model, messages=[{"role": "user", "content": prompt}],
                temperature=0.3
            )
            return res.choices[0].message.content.strip()
        except:
            return ""

    def detect_intent(self, user_message: str, prev_ans: str, summary: str = "") -> dict:
        form_list = "\n".join([f"- {k}: {v.description}" for k, v in self.forms.items()])
        prompt = INTENT_PROMPT.format(form_list=form_list, prev_sys_message=prev_ans, user_message=user_message, summary=summary)
        return self._call_json(INTENT_LLM, prompt)

    def extract_data(self, form_name: str, summary: str, user_message: str, prev_ans: str) -> dict:
        spec = self.forms[form_name]
        fields_desc = "\n".join([f"- {k}: {v.meaning}" for k, v in spec.fields.items()])

        prompt = EXTRACT_PROMPT.format(
            form_name=form_name,
            fields_desc=fields_desc,
            user_data=dict_format(MOCK_USER_DATA),
            summary=summary,
            prev_sys_message=prev_ans,
            user_message=user_message
        )
        return self._call_json(EXTRACTOR_LLM, prompt).get("values", {})

    def validate_data(self, form_name: str, extracted_data: dict) -> List[str]:
        spec = self.forms[form_name]
        errors = []

        for field_key, field_spec in spec.fields.items():
            value = extracted_data.get(field_key)

            if field_spec.required and not value:
                errors.append(f"Thiếu thông tin: {field_spec.label} ({field_spec.meaning})")
                continue

            if value and field_spec.pattern:
                if not re.match(field_spec.pattern, str(value)):
                    errors.append(f"Sai định dạng '{field_spec.label}': Giá trị '{value}' không khớp quy tắc.")

        return errors

    def check_user_confirmation(self, form_name: str, summary: str, prev_ans: str, user_message: str) -> bool:
        res = self._call_json(CHECKER_LLM, CONFIRM_CHECK_PROMPT.format(form_name=form_name, summary=summary, prev_sys_message=prev_ans, user_message=user_message))
        return res.get("confirmed", False)

    def generate_form(self, form_name: str, extracted_data: dict) -> str:
        template_path = f"{FORMS_PATH}/{form_name}.docx"
        if not Path(template_path).exists():
            return f"ERROR: Không tìm thấy file mẫu {template_path}"

        doc = DocxTemplate(template_path)
        doc.render(extracted_data)

        output_path = f"output/{form_name}.docx"
        doc.save(output_path)
        return output_path

    def process_message(self, user_message: str, summary: str, current_form_context: str = None, prev_ans: str = "") -> dict:
        summary = summary or 'Chưa có'
        form_name = current_form_context
        intent_data = self.detect_intent(user_message, prev_ans, summary)

        if intent_data.get("form_name"):
            form_name = intent_data["form_name"]

        if not form_name:
            rag_context = self.rag.search(user_message)
        else:
            rag_context = self.rag.search(user_message, 20, 1)

        if not form_name:
            chat_prompt = CHAT_PROMPT.format(
                rag_context=rag_context,
                user_data=dict_format(MOCK_USER_DATA),
                form_list = "\n".join([f"- {k}: {v.description}" for k, v in self.forms.items()]),
                summary=summary,
                prev_sys_message=prev_ans,
                user_message=user_message
            )

            answer = self._call_text(CHAT_LLM, chat_prompt)
            new_summary = self._update_summary(rag_context, summary, user_message, answer)
            return {"answer": answer, "summary": new_summary, "form_context": None, "status": "chat", "rag_context": rag_context}

        elif intent_data["is_recommend"]:
            chat_prompt = CHAT_PROMPT.format(
                rag_context=rag_context,
                user_data=dict_format(MOCK_USER_DATA),
                form_list = "\n".join([f"- {k}: {v.description}" for k, v in self.forms.items()]),
                summary=summary + f"\nHệ thống đang gợi ý rằng có thể người dùng muốn tạo đơn {form_name}",
                prev_sys_message=prev_ans,
                user_message=user_message
            )

            answer = self._call_text(CHAT_LLM, chat_prompt)
            new_summary = self._update_summary(rag_context, summary, user_message, answer)
            return {"answer": answer, "summary": new_summary, "form_context": None, "status": "chat", "rag_context": rag_context}

        spec = self.forms[form_name]

        extracted_values = self.extract_data(form_name, summary, user_message, prev_ans=prev_ans)

        errors = self.validate_data(form_name, extracted_values)

        if errors:
            field_specs_text = "\n".join([f"- {k}: {v.meaning} (Format: {v.pattern or 'Tự do'})" for k, v in spec.fields.items()])

            guidance_prompt = GUIDANCE_PROMPT.format(
                rag_context=rag_context,
                form_description=spec.description,
                current_data_json=json.dumps(extracted_values, ensure_ascii=False),
                error_list="\n".join([f"- {e}" for e in errors]),
                field_specs=field_specs_text,
                summary=summary,
                prev_sys_message=prev_ans,
                user_message=user_message
            )

            answer = self._call_text(GUIDE_LLM, guidance_prompt)
            new_summary = self._update_summary(rag_context, summary, user_message, answer)
            return {"answer": answer, "summary": new_summary, "form_context": form_name, "status": "guiding", "rag_context": rag_context}

        else:
            is_confirmed = self.check_user_confirmation(form_name, summary, prev_ans, user_message)

            if is_confirmed:
                answer = f"Đã tạo thành công biểu mẫu tại {self.generate_form(form_name, extracted_values)} theo yêu cầu của sinh viên."
                new_summary = self._update_summary(rag_context, summary, user_message, "\n" + answer)
                return {"answer": answer, "summary": new_summary, "form_context": None, "status": "done", "rag_context": rag_context}
            else:
                valid_data_view = "\n".join([f"- {spec.fields[k].label}: {v}" for k, v in extracted_values.items()])
                review_prompt = REVIEW_PROMPT.format(
                    form_description=spec.description,
                    form_name=form_name,
                    fields_desc="\n".join([f"- {k}: {v.meaning}" for k, v in spec.fields.items()]),
                    user_data=json.dumps(MOCK_USER_DATA, ensure_ascii=False),
                    valid_data_view=valid_data_view,
                    summary=summary,
                    prev_sys_message=prev_ans,
                    user_message=user_message
                )
                answer = self._call_text(GUIDE_LLM, review_prompt)
                new_summary = self._update_summary(rag_context, summary, user_message, answer)
                return {"answer": answer, "summary": new_summary, "form_context": form_name, "status": "reviewing", "rag_context": rag_context}

    def _update_summary(self, rag_context, old, user, bot):
        return self._call_text(SUM_LLM, SUMMARIZE_PROMPT.format(rag_context=rag_context, old_summary=old, user_message=user, bot_response=bot))

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
Path("output").mkdir(parents=True, exist_ok=True)

rag = HustRAG(embedder, reranker, reranker_tokenizer, retrieve_k=15, rerank_k=2)
chatbot = HustChat(forms=load_forms(), rag=rag, api_key=GROQ_API_KEY)

current_summary = ""
current_form_context = None
history = []
result = {"answer": "Chưa có"}

print("=== ChatBot Hỗ trợ Sinh viên HUST ===")
print("Gõ 'exit' để thoát.\n")

while True:
    user_input = input("Sinh viên: ")
    if user_input.lower() == "exit":
        break

    result = chatbot.process_message(
        user_message=user_input,
        summary=current_summary,
        current_form_context=current_form_context,
        prev_ans=result["answer"]
    )

    current_summary = result["summary"]
    current_form_context = result["form_context"]

    history.append({
        "user": user_input,
        "bot": result["answer"],
        "status": result["status"],
        "summary": result["summary"],
        "rag_context": result["rag_context"],
        "form": current_form_context
    })

    print(f"\nChatBot:\n{result['answer']}\n")
    print("-" * 50)

In [ ]:
for item in history:
    print("=" * 200)
    print("=" * 200)
    for key, value in item.items():
        print(f"\n=== {key.upper()} ===")
        print(value)